In [1]:
from __future__ import unicode_literals, print_function
import spacy
import en_core_web_sm
import en_core_web_md
# import en_core_web_lg
import random
from pathlib import Path
from spacy.util import minibatch, compounding
import tarfile
import xml.etree.ElementTree as ET
import os
from pandas import ExcelWriter
import pandas as pd
from sklearn.model_selection import train_test_split
import timeit
from spacy.gold import GoldParse
from spacy.scorer import Scorer

In [9]:
def transformAge(orginalData = None):
    texts = [text for text in orginalData['text']]
    transformTags = []
    for tags in orginalData['tags']:
        entity = {'entities': []}
        for tag in eval(tags):
            if tag['TYPE'] == 'AGE':
                entity['entities'].append((int(tag['start']), int(tag['end']), 'AGE'))
        transformTags.append(entity)
    transformedTrain = list(zip(texts, transformTags))
    return transformedTrain
def findAllEntityTransform(data = None):
    uniqueTag = set()
    for (_, tags) in data:
        for tag in tags['entities']:
            uniqueTag.add(tag[2])
    return uniqueTag

def main(model=None, n_iter=3, output_dir=None):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    random.seed(0)
    if model is not None:
        nlp = spacy.load(model) # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")
    for LABEL in uniqueTag:
        ner.add_label(LABEL)  # add new entity label to entity recognizer
    # Adding extraneous labels shouldn't mess anything up
#     ner.add_label("VEGETABLE")
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

    with nlp.disable_pipes(*other_pipes):  # only train NER
        sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                try:
                    nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
                except:
                    textWithProblem.append(texts)
                    pass
            print("Losses", losses)

    # test the trained model
    test_text =  test.iloc[0, 0]
    
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print('The entities:', ent.label_, ent.text)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
#         nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)
        

def evaluate(ner_model, examples):
    stats = ['ents_p', 'ents_r', 'ents_f', 'ents_per_type']
    scorer = Scorer() 
    for input_, annot in examples:
        try:
            doc_gold_text = ner_model.make_doc(input_) #the raw text
            gold = GoldParse(doc_gold_text, entities=annot['entities']) # combine the raw text with corrected id
            pred_value = ner_model(input_) # a doc object, which contains the predicted entity
            scorer.score(pred_value, gold) # calculate the accuracy
        except:
            textWithProblem.append((input_, annot))
    return {k: scorer.scores[k] for k in stats} # accumulate all the documents

# find all entity type in the original data
def findAllEntity(data = None):
    uniqueTag = set()
    [[uniqueTag.add(tag['TYPE']) for tag in eval(tags)] for tags in data['tags']]
    return uniqueTag

def findOneEntity(data = None, entType = None):
    for tags in data['tags']:
        for tag in eval(tags):
            if tag['TYPE'] == entType:
                print(tag)

In [43]:
a = transformAge(df_2006Test)

In [47]:
for i in a:
    if i[1]['entities']:
        print(i[0],i[1], i[0][533:537])
        break



981917876
FIH
7380847
80436/9qbu
552080
10/11/1993 12:00:00 AM
CEREBELLAR HEMATOMA .
Unsigned
DIS
Report Status :
Unsigned
ADMISSION DATE :
10-11-93
DISCHARGE DATE :
11-4-93
PRINCIPAL DIAGNOSIS :
cerebellar hematoma .
ASSOCIATED DIAGNOSIS :
1. deep venous thrombosis ,
2. history of chronic obstructive pulmonary disease .
SPECIAL PROCEDURES AND OPERATIONS :
10-11-93 , suboccipital craniectomy with evacuation of cerebellar hematoma and left lateral cerebellar hemisphere resection .
HISTORY OF PRESENT ILLNESS :
The patient is a 112y year old female who was found unresponsive and underwent an initial evaluation at Tonland County Medical Center where a computerized tomography scan showed evidence of an acute posterior ______________ bleed .
She was transferred to the Fairm of Ijordcompmac Hospital for further neurosurgical evaluation .
PAST MEDICAL HISTORY :
As documented previously in the patient 's medical record , includes a history of chronic obstructive pulmonary disease , and corona

In [41]:
findAllEntity(df_2006Train)

{'AGE', 'DATE', 'DOCTOR', 'HOSPITAL', 'ID', 'LOCATION', 'PATIENT', 'PHONE'}

In [42]:
df = pd.read_excel('./PythonExport.xlsx')
findAllEntity(df)

{'AGE',
 'BIOID',
 'CITY',
 'COUNTRY',
 'DATE',
 'DEVICE',
 'DOCTOR',
 'EMAIL',
 'FAX',
 'HEALTHPLAN',
 'HOSPITAL',
 'IDNUM',
 'LOCATION-OTHER',
 'MEDICALRECORD',
 'ORGANIZATION',
 'PATIENT',
 'PHONE',
 'PROFESSION',
 'STATE',
 'STREET',
 'URL',
 'USERNAME',
 'ZIP'}

In [30]:
df.iloc[0, 1]

"[{'id': 'P0', 'start': '16', 'end': '26', 'text': '2106-02-12', 'TYPE': 'DATE', 'comment': ''}, {'id': 'P1', 'start': '28', 'end': '36', 'text': 'Campbell', 'TYPE': 'HOSPITAL', 'comment': ''}, {'id': 'P2', 'start': '59', 'end': '74', 'text': '4 Madera Circle', 'TYPE': 'STREET', 'comment': ''}, {'id': 'P3', 'start': '75', 'end': '79', 'text': 'Omak', 'TYPE': 'CITY', 'comment': ''}, {'id': 'P4', 'start': '81', 'end': '83', 'text': 'GA', 'TYPE': 'STATE', 'comment': ''}, {'id': 'P5', 'start': '84', 'end': '89', 'text': '28172', 'TYPE': 'ZIP', 'comment': ''}, {'id': 'P6', 'start': '92', 'end': '108', 'text': 'Habib Valenzuela', 'TYPE': 'DOCTOR', 'comment': ''}, {'id': 'P7', 'start': '164', 'end': '182', 'text': 'Valdez, Harlan Jr.', 'TYPE': 'PATIENT', 'comment': ''}, {'id': 'P8', 'start': '228', 'end': '239', 'text': '845-41-54-4', 'TYPE': 'MEDICALRECORD', 'comment': ''}, {'id': 'P9', 'start': '285', 'end': '302', 'text': 'February 12, 2106', 'TYPE': 'DATE', 'comment': ''}, {'id': 'P10', '

In [8]:
df = pd.read_excel('./PythonExport.xlsx')
df_2006Train = pd.read_excel('./i2b2-2006Train.xlsx')
df_2006Test = pd.read_excel('./i2b2-2006Test.xlsx')
df = pd.concat([df, df_2006Test, df_2006Train], axis = 0)
train, test = train_test_split(df, test_size=0.1, random_state = 42)
transformedTrainAge = transformAge(train)
uniqueTag = findAllEntityTransform(transformedTrainAge)
TRAIN_DATA = transformedTrainAge
textWithProblem = []

In [39]:
df_2006Train.iloc[0, 1]

"[{'start': '2', 'end': '11', 'text': '123547445', 'TYPE': 'ID'}, {'start': '12', 'end': '15', 'text': 'FIH', 'TYPE': 'HOSPITAL'}, {'start': '16', 'end': '23', 'text': '7111426', 'TYPE': 'ID'}, {'start': '24', 'end': '34', 'text': '47933/f911', 'TYPE': 'ID'}, {'start': '35', 'end': '41', 'text': '557344', 'TYPE': 'ID'}, {'start': '42', 'end': '47', 'text': '11/19', 'TYPE': 'DATE'}, {'start': '42', 'end': '47', 'text': '11/19', 'TYPE': 'DATE'}, {'start': '164', 'end': '169', 'text': '11/28', 'TYPE': 'DATE'}, {'start': '275', 'end': '280', 'text': 'Blind', 'TYPE': 'PATIENT'}, {'start': '442', 'end': '455', 'text': 'November 13th', 'TYPE': 'DATE'}, {'start': '459', 'end': '477', 'text': 'Sephsandpot Center', 'TYPE': 'HOSPITAL'}, {'start': '514', 'end': '527', 'text': 'November 15th', 'TYPE': 'DATE'}, {'start': '599', 'end': '642', 'text': 'Valtawnprinceel Community Memorial Hospital', 'TYPE': 'HOSPITAL'}, {'start': '682', 'end': '698', 'text': '16th of November', 'TYPE': 'DATE'}, {'start'

In [52]:
df.head()

,text,tags
0,\n\n\nRecord date: 2106-02-12\n\nCampbell Orth...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
1,\n\n\nRecord date: 2108-03-14\n\n ...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
2,\n\n\nRecord date: 2109-09-14\n\n \nSeptember ...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
3,\n\n\nRecord date: 2111-10-10\n\n \n\n\n\nC...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
4,\n\n\nRecord date: 2111-12-14\n\n \n\n\n\nNEUR...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."


In [53]:
df_2006Test.head()

,text,tags
0,\n\n113416550\nPRGH\n13523357\n630190\n6/7/199...,"[{'start': '2', 'end': '11', 'text': '11341655..."
1,\n\n229937784\nFIH\n8765992\n87919/3984\n75471...,"[{'start': '2', 'end': '11', 'text': '22993778..."
2,\n\n010760828\nFIH\n4604997\n76732/99w3\n50342...,"[{'start': '2', 'end': '11', 'text': '01076082..."
3,\n\n333145593\nFIH\n9229891\n27487/f4b3\n69921...,"[{'start': '2', 'end': '11', 'text': '33314559..."
4,\n\n333145593\nFIH\n9229891\n50096/c65m\n57163...,"[{'start': '2', 'end': '11', 'text': '33314559..."


In [51]:
i = 0
for doc in transformedTrainAge:
    text, ent = doc
    if not ent['entities']:
        print(text)
    



855934190 DH
4656647
94459550
87554035
12/13/2006 12:00:00 AM
Face Sheet / Discharge Order
SIGNED
DIS
[ report_end ]




712485163
HLGMC
8634097
28531/9v89
9/8/1994 12:00:00 AM
MYOCARDIAL INFARCTION .
Unsigned
DIS
Report Status :
Unsigned
ADMISSION DATE :
9/8/94
DISCHARGE DATE :
9/10/94
PRINCIPAL DIAGNOSIS :
Myocardial infarction .
ASSOCIATED DIAGNOSIS :
Coronary artery disease .
2. Hiatal hernia .
3. Anxiety .
SPECIAL PROCEDURES AND OPERATIONS :
1. Coronary angioplasty ( 9/9/94 ) .
DOCTORS DISCHARGE ORDERS AND MEDICATIONS :
1. Zantac , 150 mg. , po , b.i.d.
2. Isordil , 10 mg. , po , t.i.d.
3. Ecotrin , 325 mg. , po , qD.
4. Nitroglycerin , 0.4 mg. , sublingual , prn.
HISTORY OF PRESENT ILLNESS :
Mr. Little is a 53 year old male who is under the care of Dr. Royendchaelmares , at Hend Geadcoastcar Hospital , with the diagnosis of coronary artery disease .
He has a history of an old inferior myocardial infarction .
He was well until three days prior to admission , when he developed an




892701093
PUOMC
3656771
679970
055111
3/21/1997 12:00:00 AM
STAGE III LUNG CARCINOMA .
Unsigned
DIS
Report Status :
Unsigned
DISCHARGE SUMMARY
NAME :
STIRE , BRIETHAN B
UNIT NUMBER :
751-51-51
ADMISSION DATE :
03/21/97
DISCHARGE DATE :
06/02/97
PRINCIPAL DIAGNOSIS :
Stage III lung carcinoma .
ASSOCIATED DIAGNOSIS :
Right upper lobectomy April 14 , 1997 .
HISTORY OF PRESENT ILLNESS :
Mr. Stire is a sixty nine-year-old male with an eighty pack year tobacco history noted to have a mass in the right upper lobe .
The patient underwent a right upper lobectomy on February 11 , 1997 and returned for a left lower lobectomy .
He is status post chemotherapy in October 1997 .
The patient had an unremarkable postoperative course in January .
He walks daily and resumed exercises , treadmill and stationary bike .
Denied weight loss , cough , shortness of breath or hemoptysis .
PAST MEDICAL HISTORY :
Right upper lobectomy February 11 , 1997 .
Three vessel coronary artery bypass graft April 1993 wit





Record date: 2062-02-10

RANKIN GENERAL HOSPITAL
Arthritis Center
39 Autumn Glen Circle                             Rivers, Ida
Brazzaville, UT  33953                             71701456
403-5583                                      2-10-62
 
 
 
1.   Sjogren's.
2.   Diabetes.
 
CURRENT MEDICATIONS:  Insulin, estrogens, Tears.
 
HPI:  Is noting a dry cough.  Has noted it over the last couple of 
months.  It has been intermittent.  No fever.
 
Using artificial tears.  Not using Saliva.  Diabetes, she reports, under 
excellent control.  However, is beginning to lose weight.
 
No extraglandular symptoms now.
 
PHYSICAL EXAMINATION:  Weight 138.  Oral cavity dry.  Neck: supple 
without adenopathy.  Lungs: clear.  Cardiac exam: no murmurs.  Abdomen: 
soft, nontender, without spleen tip.  Joints: no active synovitis.  
Proximal muscle strength normal.
 
ASSESSMENT:
1.   Primary Sjogren's, stable.
2.   Diabetes.
 
PLAN:
1.   Tears and Saliva.
2.   Dietary intake encouraged.
3.   Bone den




745232329
PUOMC
9543139
96458/7hv1
139846
9/14/1993 12:00:00 AM
Discharge Summary
Unsigned
DIS
Report Status :
Unsigned
ADMISSION DATE :
9/14/93
DISCHARGE DATE :
9/15/93
CHIEF COMPLAINT :
left axillary node dissection , status post breast carcinoma .
HISTORY OF PRESENT ILLNESS :
Patient is a 49 year old previously healthy female , who was diagnosed in June , 1992 , with a breast carcinoma , after she presented with some breast discomfort on the left side .
She underwent a wide resection of a left breast mass by Dr. Ausguall on 8/17/93 .
Pathology of this resection revealed a Grade III / III infiltrating ductal carcinoma .
She now presents for left axillary node dissection , to complete her staging .
She has no complaints at the time of presentation .
REVIEW OF SYSTEMS :
noncontributory , her last menstrual period was regular , and last week .
MEDICATIONS ON ADMISSION :
:w
:q
none .
PAST MEDICAL HISTORY :
Noncontributory .
ALLERGIES :
no known drug allergies .
SOCIAL HISTORY :
Denies




110874821 WGH
9627305
417341
6106510
1/3/2005 12:00:00 AM
ED Discharge Summary
Unsigned
DIS
Report Status :
Unsigned
ED DISCHARGE NOTIFICATION / SUMMARY
BOORMDRUDD , ZIERA
MRN :
9627305
Age :
72y
REGISTRATION DATE :
01/03/2005 02:50 PM
Provider :
MAAAREA OBEROC
PRELIMINARY REPORT
Benefits Assigned :
Y
Discharge Note Date / Time :
01/03/2005 18:41
Discharge Status :
Discharged
Condition on Discharge :
Stable
Patient States Complaint :
LAC TO HEAD
Standardized Discharge Instructions :
The patient was given printed instructions for wound care ( English ) .
Diagnosis :
Laceration to scalp
Treatment Rendered :
Exam , Wound closed with staples
Discharge Medications :
NONE
Follow up Service :
It is recommended that you receive follow-up care in the D Valley Health with Dr Oberoc on December 15th .
Please call 350-059-2924 to schedule an appt .
Appointments are scheduled for early morning .
If you have a managed care plan , check with your PCP before making this appt
Disposition , Follow up

...




985213692 WGH
8894679
77279
1063544
8/14/2003 12:00:00 AM
OB Discharge Summary
Signed
DIS
Report Status :
Signed OB EMR L and D
DISCHARGE SUMMARY
NAME :
BETLA CHECK
UNIT NUMBER :
393-00-85
ADMISSION DATE :
20030814
DISCHARGE DATE :
20030816
PRINCIPAL DISCHARGE DIAGNOSIS :
Normal Vaginal Delivery
ASSOCIATED DIAGNOSES :
Acute Urinary Tract Infection ; Heart Murmur , history of
PRINCIPAL PROCEDURE OR OPERATION :
Spontaneous Vertex Vaginal Delivery
ASSOCIATED PROCEDURES OR OPERATIONS :
POSTPARTUM DIAGNOSTIC PROCEDURES :
None
POSTPARTUM THERAPEUTIC PROCEDURES :
None
OTHER POSTPARTUM THERAPIES :
Routine Post Partum Care
HISTORY AND REASON FOR HOSPITALIZATION :
Active Labor
PHYSICAL EXAMINATION :
HEIGHT 61 HEENT NORMAL MOUTH NORMAL NECK NORMAL BREASTS right breast 11 oclock , left breast , 11o clock NIPPLES NORMAL CHEST NORMAL COR ( + ) diastolic murmur ABDOMEN NORMAL Gravid , FHR 130-140s good variability without decels , neg .
CST EXTREM NORMAL SKIN NORMAL NODES NORMAL VULVA NORMAL VAGIN




677168937 RWH
9049156
280494
763694
11/05/2001 12:00:00 AM
ED Discharge Summary
Unsigned
DIS
Report Status :
Unsigned
ED DISCHARGE NOTIFICATION / SUMMARY
LOAN , LYKEY
MRN :
9049156
Age :
54y
REGISTRATION DATE :
11/05/2001 11:11 AM
Provider :
Kema Cuchkote
PRELIMINARY REPORT
Discharge Date / Time :
11/05/2001 13:38
Discharge Status :
Discharged
Condition on Discharge :
Stable
Patient States Complaint :
SCAPULA PAIN
Standardized Discharge Instructions :
The patient was given printed instructions for chest pain ( English ) .
The patient was given printed instructions for chest wall injury ( English ) .
Diagnosis :
Chest wall strain
Treatment Rendered :
motrin and tylenol
Discharge Medications :
naprosyn 550 mg by mouth twice a day WITH FOOD ; if it hurts your stomach , cut-dose in half and call your doctor .
Also take tylenol 2 pills every 8 hours for more pain .
valium for sleep at night .
Follow up Service :
if not getting better by the end of this upcoming week , make sure you see a




240417784 EH
48067463
9466660
7/30/2005 12:00:00 AM
hypoglycemia
DIS
Admission Date :
07/30/2005
Report Status :
Discharge Date :
07/31/2005
****** DISCHARGE ORDERS ******
AGNEAGURNE , TOFRAN 566-00-27-6 N83
Room :
97H-228
Service :
MED
DISCHARGE PATIENT ON :
07/31/05 AT 04:00 PM
CONTINGENT UPON
Attending evaluation
WILL D / C ORDER BE USED AS THE D / C SUMMARY :
YES
Attending :
JESC , ON , M.D.
CODE STATUS :
Full code
DISPOSITION :
Home
DISCHARGE MEDICATIONS :
ACETYLSALICYLIC ACID 81 MG PO QD DIOVAN ( VALSARTAN ) 160 MG PO QD
Number of Doses Required ( approximate ) :
4 NEXIUM ( ESOMEPRAZOLE ) 20 MG PO QD LANTUS ( INSULIN GLARGINE ) 12 UNITS SC QHS METFORMIN 1,000
MG PO BID
DIET :
No Restrictions
ACTIVITY :
Resume regular exercise
FOLLOW UP APPOINTMENT ( S ) :
Dr. Trial 1 week , Diabetes Nurse- Night .
Dral And Man Valley Hospital 007-151-8823 Aug 2th , 2:20pm ,
ALLERGY :
SIMVASTATIN , ACE Inhibitor
ADMIT DIAGNOSIS :
hypoglycemia
PRINCIPAL DISCHARGE DIAGNOSIS ;
Responsible After St




932057504 DH
5898830
809200
356490
11/25/2000 12:00:00 AM
ED Discharge Summary
Unsigned
DIS
Report Status :
Unsigned
ED DISCHARGE NOTIFICATION
AGACHBRAUCH , RENSIAANARC
MRN :
5898830
Age :
44y
REGISTRATION DATE :
11/25/2000 01:36 AM
PRELIMINARY REPORT
This is to notify you that your patient , AGACHBRAUCH , RENSIAANARC arrived in the Emergency Department at Daughtersvillshamen's Hospital on 11/25/2000 01:36 AM .
The patient presented with a chief complaint of AF-RESOLVED .
Following evaluation and treatment , the patient 's disposition at the end of the visit was TREATED AND RELEASED .
If you need additional information please call 613-870-0699 .
[ report_end ]




IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [50]:
i

962

In [25]:
train.iloc[2, 0].find

'\n\n894281954\nCTMC\n03528277\n385056\n1/22/1997 12:00:00 AM\nRESIDUAL SKULL BASE CHORDOMA\nSigned\nDIS\nAdmission Date :\n01/22/1997\nReport Status :\nSigned\nDischarge Date :\n02/06/1997\nPRINCIPAL DIAGNOSIS :\nCHORDOMA .\nPROCEDURE :\n1/22/97 Right suboccipital craniotomy with microresection of tumor with laminectomy .\nHISTORY OF PRESENT ILLNESS :\nMr. Marker is a 60 year-old man presented with a two year history with right neck pain .\nAn initial CT scan was negative , however , eventually a scan in August , 1996 showed a right wedge skull base mass .\nIn September , 1997 he underwent cerebral angiography complicated by an embolic cerebrovascular accident which presented with confusion in an right homonymous hemianopsia .\nHe was taken to the Intensive Care Unit .\nA follow-up MRI showed low attenuation lesions in both occipital lobes consistent with embolic stroke .\nHe was treated with aspirin .\nNovember 19 , 1997 , he underwent a resection of a skull base chordoma and a later

In [38]:
transformedTrainAge[5][1]

{'entities': [(236, 238, 'AGE')]}

In [37]:
train.iloc[5, 1]

"[{'id': 'P0', 'start': '16', 'end': '26', 'text': '2076-04-24', 'TYPE': 'DATE', 'comment': ''}, {'id': 'P1', 'start': '36', 'end': '44', 'text': '91464150', 'TYPE': 'MEDICALRECORD', 'comment': ''}, {'id': 'P2', 'start': '45', 'end': '57', 'text': 'Godwin, Yair', 'TYPE': 'PATIENT', 'comment': ''}, {'id': 'P3', 'start': '58', 'end': '66', 'text': '04/24/76', 'TYPE': 'DATE', 'comment': ''}, {'id': 'P4', 'start': '67', 'end': '87', 'text': 'MURRAY, MARION OSCAR', 'TYPE': 'DOCTOR', 'comment': ''}, {'id': 'P5', 'start': '236', 'end': '238', 'text': '70', 'TYPE': 'AGE', 'comment': ''}, {'id': 'P6', 'start': '270', 'end': '284', 'text': 'Bowne Hospital', 'TYPE': 'HOSPITAL', 'comment': ''}, {'id': 'P7', 'start': '3534', 'end': '3540', 'text': 'Rivers', 'TYPE': 'DOCTOR', 'comment': ''}, {'id': 'P8', 'start': '3830', 'end': '3850', 'text': 'MURRAY, MARION OSCAR', 'TYPE': 'DOCTOR', 'comment': ''}, {'id': 'P9', 'start': '3860', 'end': '3868', 'text': '04/24/76', 'TYPE': 'DATE', 'comment': ''}, {'i

In [35]:
print(transformedTrainAge[2][0])



894281954
CTMC
03528277
385056
1/22/1997 12:00:00 AM
RESIDUAL SKULL BASE CHORDOMA
Signed
DIS
Admission Date :
01/22/1997
Report Status :
Signed
Discharge Date :
02/06/1997
PRINCIPAL DIAGNOSIS :
CHORDOMA .
PROCEDURE :
1/22/97 Right suboccipital craniotomy with microresection of tumor with laminectomy .
HISTORY OF PRESENT ILLNESS :
Mr. Marker is a 60 year-old man presented with a two year history with right neck pain .
An initial CT scan was negative , however , eventually a scan in August , 1996 showed a right wedge skull base mass .
In September , 1997 he underwent cerebral angiography complicated by an embolic cerebrovascular accident which presented with confusion in an right homonymous hemianopsia .
He was taken to the Intensive Care Unit .
A follow-up MRI showed low attenuation lesions in both occipital lobes consistent with embolic stroke .
He was treated with aspirin .
November 19 , 1997 , he underwent a resection of a skull base chordoma and a lateral mandibulotomy with trache

In [10]:
findOneEntity(df, 'AGE')

{'id': 'P11', 'start': '313', 'end': '315', 'text': '43', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P5', 'start': '509', 'end': '511', 'text': '45', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P14', 'start': '352', 'end': '354', 'text': '46', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P16', 'start': '1023', 'end': '1025', 'text': '15', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P17', 'start': '1045', 'end': '1047', 'text': '10', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P7', 'start': '317', 'end': '319', 'text': '48', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P9', 'start': '3495', 'end': '3497', 'text': '48', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P6', 'start': '202', 'end': '204', 'text': '49', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P7', 'start': '317', 'end': '319', 'text': '49', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P14', 'start': '2676', 'end': '2678', 'text': '17', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P15', 'start': '2691', 'end': '2693', 'text': '13', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P16', 'start': '277

{'id': 'P12', 'start': '260', 'end': '262', 'text': '61', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P1', 'start': '59', 'end': '61', 'text': '61', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P8', 'start': '358', 'end': '360', 'text': '53', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P16', 'start': '3152', 'end': '3154', 'text': '24', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P17', 'start': '3159', 'end': '3161', 'text': '26', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P20', 'start': '3396', 'end': '3398', 'text': '59', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P31', 'start': '8160', 'end': '8162', 'text': '53', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P2', 'start': '137', 'end': '139', 'text': '53', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P7', 'start': '210', 'end': '212', 'text': '53', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P16', 'start': '6088', 'end': '6090', 'text': '53', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P1', 'start': '78', 'end': '80', 'text': '53', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P2', 'start': '126', 'e

{'id': 'P11', 'start': '2094', 'end': '2096', 'text': '80', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P6', 'start': '361', 'end': '363', 'text': '81', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P21', 'start': '2875', 'end': '2877', 'text': '80', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P4', 'start': '135', 'end': '137', 'text': '59', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P11', 'start': '2727', 'end': '2729', 'text': '59', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P3', 'start': '107', 'end': '109', 'text': '59', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P12', 'start': '2315', 'end': '2317', 'text': '59', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P3', 'start': '110', 'end': '112', 'text': '60', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P19', 'start': '3187', 'end': '3189', 'text': '60', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P10', 'start': '408', 'end': '410', 'text': '83', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P7', 'start': '211', 'end': '213', 'text': '83', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P6', 'start': '188'

{'id': 'P43', 'start': '12285', 'end': '12287', 'text': '55', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P44', 'start': '12297', 'end': '12299', 'text': '65', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P3', 'start': '119', 'end': '121', 'text': '68', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P12', 'start': '2185', 'end': '2187', 'text': '89', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P13', 'start': '2264', 'end': '2267', 'text': '40s', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P3', 'start': '120', 'end': '122', 'text': '69', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P5', 'start': '624', 'end': '626', 'text': '69', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P14', 'start': '6174', 'end': '6176', 'text': '69', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P3', 'start': '86', 'end': '88', 'text': '76', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P9', 'start': '1901', 'end': '1903', 'text': '76', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P9', 'start': '2230', 'end': '2232', 'text': '76', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P2', 'start':

{'id': 'P15', 'start': '422', 'end': '424', 'text': '56', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P18', 'start': '1065', 'end': '1067', 'text': '56', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P19', 'start': '1708', 'end': '1710', 'text': '56', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P4', 'start': '168', 'end': '170', 'text': '59', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P5', 'start': '246', 'end': '248', 'text': '59', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P7', 'start': '295', 'end': '297', 'text': '62', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P5', 'start': '235', 'end': '237', 'text': '38', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P5', 'start': '224', 'end': '226', 'text': '41', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P11', 'start': '289', 'end': '291', 'text': '43', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P6', 'start': '244', 'end': '246', 'text': '55', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P11', 'start': '2166', 'end': '2168', 'text': '81', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P12', 'start': '2304', '

...

{'id': 'P7', 'start': '544', 'end': '546', 'text': '44', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P23', 'start': '7557', 'end': '7559', 'text': '44', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P6', 'start': '312', 'end': '314', 'text': '47', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P18', 'start': '5163', 'end': '5165', 'text': '47', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P1', 'start': '63', 'end': '65', 'text': '70', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P15', 'start': '326', 'end': '328', 'text': '72', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P6', 'start': '277', 'end': '279', 'text': '74', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P16', 'start': '3853', 'end': '3855', 'text': '62', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P17', 'start': '3889', 'end': '3891', 'text': '58', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P20', 'start': '6790', 'end': '6792', 'text': '74', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P27', 'start': '11479', 'end': '11481', 'text': '74', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P6', 'start': '4

{'id': 'P15', 'start': '8320', 'end': '8322', 'text': '53', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P7', 'start': '2595', 'end': '2597', 'text': '54', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P4', 'start': '87', 'end': '89', 'text': '28', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P2', 'start': '139', 'end': '141', 'text': '29', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P5', 'start': '424', 'end': '426', 'text': '29', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P6', 'start': '537', 'end': '545', 'text': '9 months', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P7', 'start': '930', 'end': '935', 'text': '9 mos', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P1', 'start': '49', 'end': '51', 'text': '30', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P4', 'start': '862', 'end': '867', 'text': '9 mos', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P3', 'start': '94', 'end': '96', 'text': '32', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P9', 'start': '1797', 'end': '1799', 'text': '17', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P11', 'start': '372', 

{'id': 'P16', 'start': '1902', 'end': '1903', 'text': '9', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P17', 'start': '1909', 'end': '1911', 'text': '11', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P20', 'start': '2458', 'end': '2460', 'text': '39', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P5', 'start': '369', 'end': '371', 'text': '41', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P8', 'start': '2089', 'end': '2091', 'text': '41', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P5', 'start': '264', 'end': '266', 'text': '41', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P18', 'start': '4098', 'end': '4100', 'text': '41', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P6', 'start': '291', 'end': '293', 'text': '74', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P12', 'start': '2439', 'end': '2441', 'text': '63', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P13', 'start': '2481', 'end': '2483', 'text': '47', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P14', 'start': '2557', 'end': '2561', 'text': "70's", 'TYPE': 'AGE', 'comment': ''}
{'id': 'P15', 'star

{'id': 'P14', 'start': '4219', 'end': '4221', 'text': '77', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P3', 'start': '268', 'end': '270', 'text': '44', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P7', 'start': '1098', 'end': '1100', 'text': '25', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P18', 'start': '2027', 'end': '2029', 'text': '11', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P33', 'start': '4191', 'end': '4193', 'text': '44', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P6', 'start': '407', 'end': '409', 'text': '46', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P7', 'start': '2343', 'end': '2345', 'text': '46', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P7', 'start': '267', 'end': '269', 'text': '45', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P9', 'start': '729', 'end': '731', 'text': '25', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P10', 'start': '1747', 'end': '1749', 'text': '45', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P6', 'start': '148', 'end': '150', 'text': '48', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P47', 'start': '710

{'id': 'P6', 'start': '201', 'end': '203', 'text': '79', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P7', 'start': '424', 'end': '426', 'text': '79', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P28', 'start': '3647', 'end': '3656', 'text': '13 months', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P30', 'start': '7315', 'end': '7317', 'text': '79', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P6', 'start': '142', 'end': '144', 'text': '81', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P6', 'start': '453', 'end': '455', 'text': '63', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P6', 'start': '2274', 'end': '2277', 'text': '40s', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P4', 'start': '2627', 'end': '2630', 'text': '40s', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P6', 'start': '4801', 'end': '4803', 'text': '40', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P7', 'start': '4804', 'end': '4806', 'text': '64', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P10', 'start': '199', 'end': '201', 'text': '65', 'TYPE': 'AGE', 'comment': ''}
{'id': 'P1', 'start

In [6]:
findAllEntity(df)

{'AGE',
 'BIOID',
 'CITY',
 'COUNTRY',
 'DATE',
 'DEVICE',
 'DOCTOR',
 'EMAIL',
 'FAX',
 'HEALTHPLAN',
 'HOSPITAL',
 'ID',
 'IDNUM',
 'LOCATION',
 'LOCATION-OTHER',
 'MEDICALRECORD',
 'ORGANIZATION',
 'PATIENT',
 'PHONE',
 'PROFESSION',
 'STATE',
 'STREET',
 'URL',
 'USERNAME',
 'ZIP'}